In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from scipy.spatial import distance as dist
from sklearn.metrics import accuracy_score
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

path_train = "C:/Users/ico/3D Objects/patterndata/train"
path_test = "C:/Users/ico/3D Objects/patterndata/test"

train_data = []
train_labels = []
test_data = []
test_labels = []

# Define your class labels
class_labels = {
    "classA": 0,
    "classB": 1,
    # Add more class labels if necessary
}

# Load and label training images
for class_folder in os.listdir(path_train):
    class_path = os.path.join(path_train, class_folder)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            train_data.append(image_path)
            train_labels.append(class_labels[class_folder])

# Load and label test images
for class_folder in os.listdir(path_test):
    class_path = os.path.join(path_test, class_folder)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            test_data.append(image_path)
            test_labels.append(class_labels[class_folder])

num_pts = 16
radius = 2

lbp_train_features = []

for image_path in train_data:
    trainimg = Image.open(image_path)
    trainarr = np.asarray(trainimg)
    traingray = cv2.cvtColor(trainarr, cv2.COLOR_BGR2GRAY)
    train_lbp = local_binary_pattern(traingray, num_pts, radius, 'uniform')
    lbp_train_features.append(train_lbp)

train_images = []
for train_feature in lbp_train_features:
    train_images.append(np.expand_dims(train_feature, axis=2))
train_images = np.array(train_images)

train_labels = np.array(train_labels)

lbp_test_features = []

for image_path_test in test_data:
    testimg = Image.open(image_path_test)
    testarr = np.asarray(testimg)
    testgray = cv2.cvtColor(testarr, cv2.COLOR_BGR2GRAY)
    lbp_test = local_binary_pattern(testgray, num_pts, radius, 'uniform')
    lbp_test_features.append(lbp_test)

test_images = []
for test_feature in lbp_test_features:
    test_images.append(np.expand_dims(test_feature, axis=2))
test_images = np.array(test_images)

test_labels = np.array(test_labels)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(train_images.shape[1:])))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model withearly stopping
history = model.fit(train_images, train_labels, epochs=100, validation_data=(test_images, test_labels), callbacks=[early_stopping])

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Test accuracy:", test_acc)


Epoch 1/100
22/22 [==============================] - 32s 1s/step - loss: 10.6942 - accuracy: 0.6715 - val_loss: 0.6555 - val_accuracy: 0.6418
Epoch 2/100
22/22 [==============================] - 37s 2s/step - loss: 0.6238 - accuracy: 0.6890 - val_loss: 0.6468 - val_accuracy: 0.6418
Epoch 3/100
22/22 [==============================] - 37s 2s/step - loss: 0.5679 - accuracy: 0.7093 - val_loss: 0.6469 - val_accuracy: 0.6418
Epoch 4/100
22/22 [==============================] - 33s 1s/step - loss: 0.4508 - accuracy: 0.7878 - val_loss: 0.5903 - val_accuracy: 0.6418
Epoch 5/100
22/22 [==============================] - 35s 2s/step - loss: 0.2683 - accuracy: 0.8939 - val_loss: 1.0543 - val_accuracy: 0.4776
Epoch 6/100
22/22 [==============================] - 35s 2s/step - loss: 0.1311 - accuracy: 0.9491 - val_loss: 2.2974 - val_accuracy: 0.3582
Epoch 7/100
22/22 [==============================] - 34s 2s/step - loss: 0.0949 - accuracy: 0.9608 - val_loss: 1.7834 - val_accuracy: 0.4478
3/3 - 1s - l

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from skimage.feature import local_binary_pattern
from tensorflow.keras.callbacks import EarlyStopping

# Function to extract LBP features from an image
def extract_lbp_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 257), range=(0, 256))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

# Function to preprocess the dataset
def preprocess_dataset(images, labels):
    preprocessed_images = []
    for image in images:
        lbp_features = extract_lbp_features(image)
        preprocessed_images.append(lbp_features)
    return np.array(preprocessed_images), np.array(labels)

# Define class labels
class_labels = {
    "classA": 0,
    "classB": 1,
    # Add more class labels if necessary
}

# Load and label training images
train_data = []
train_labels = []
path_train = "C:/Users/ico/3D Objects/patterndata/train"


for class_folder in os.listdir(path_train):
    class_path = os.path.join(path_train, class_folder)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            train_data.append(cv2.imread(image_path))
            train_labels.append(class_labels[class_folder])

# Load and label test images
test_data = []
test_labels = []
path_test = "C:/Users/ico/3D Objects/patterndata/test"

for class_folder in os.listdir(path_test):
    class_path = os.path.join(path_test, class_folder)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            test_data.append(cv2.imread(image_path))
            test_labels.append(class_labels[class_folder])

# Preprocess the dataset
preprocessed_train_images, preprocessed_train_labels = preprocess_dataset(train_data, train_labels)
preprocessed_test_images, preprocessed_test_labels = preprocess_dataset(test_data, test_labels)


model = tf.keras.models.Sequential()
model.add(layers.Input(shape=(256,)))  # LBP feature vector length is 256
model.add(layers.Reshape((16,16,1)))  # Reshape input to match Dense layer
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(preprocessed_train_images, preprocessed_train_labels,
                    epochs=100, validation_data=(preprocessed_test_images, preprocessed_test_labels),
                    callbacks=[early_stopping],)

# Evaluate the model
test_loss, test_acc = model.evaluate(preprocessed_test_images, preprocessed_test_labels, verbose=2)
print("Test accuracy:", test_acc)

Epoch 1/100
22/22 [==============================] - 1s 16ms/step - loss: 0.6651 - accuracy: 0.6890 - val_loss: 0.6535 - val_accuracy: 0.6418
Epoch 2/100
22/22 [==============================] - 0s 7ms/step - loss: 0.6207 - accuracy: 0.6890 - val_loss: 0.6573 - val_accuracy: 0.6418
Epoch 3/100
22/22 [==============================] - 0s 8ms/step - loss: 0.6189 - accuracy: 0.6890 - val_loss: 0.6600 - val_accuracy: 0.6418
Epoch 4/100
22/22 [==============================] - 0s 8ms/step - loss: 0.6165 - accuracy: 0.6890 - val_loss: 0.6592 - val_accuracy: 0.6418
3/3 - 0s - loss: 0.6592 - accuracy: 0.6418 - 28ms/epoch - 9ms/step
Test accuracy: 0.641791045665741
